In [5]:
!pip install efficientnet-pytorch albumentations opencv-python pillow captum timm


  Using cached efficientnet_pytorch-0.7.1-py3-none-any.whl
  Obtaining dependency information for captum from https://files.pythonhosted.org/packages/e1/76/b21bfd2c35cab2e9a4b68b1977f7488c246c8cffa31e3361ee7610e8b5af/captum-0.7.0-py3-none-any.whl.metadata
  Using cached captum-0.7.0-py3-none-any.whl.metadata (26 kB)
  Obtaining dependency information for timm from https://files.pythonhosted.org/packages/19/0d/57fe21d3bcba4832ed59bc3bf0f544e8f0011f8ccd6fd85bc8e2a5d42c94/timm-1.0.3-py3-none-any.whl.metadata
  Using cached timm-1.0.3-py3-none-any.whl.metadata (43 kB)
  Obtaining dependency information for opencv-python-headless>=4.1.1 from https://files.pythonhosted.org/packages/03/a1/ff49de25dc4e4a16f6b9f57d7ed302acabb450e92054a5be5ecc93e66fca/opencv_python_headless-4.10.0.82-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python_headless-4.10.0.82-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Obtaining dependency information for mkl<=2021.4.0,>=2021.1.1 from https://files.pythonhost

## Loading Dataset & File paths

In [5]:
import os
import cv2
from PIL import Image
from torchvision.datasets import ImageFolder
import numpy as np
from torchvision import transforms
import albumentations as A
from torch import nn
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import torch
import random
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch.nn.functional as F
import timm

#Kaggle Dataset path
dataset='Celeb-Df-v2'

#fake and real videos path
real_videos_dir = os.path.join(dataset, 'Celeb-real')
youtube_real_dir = os.path.join(dataset,'YouTube-real')
fake_videos_dir = os.path.join(dataset, 'Celeb-synthesis')

#test videos path
test_dir= os.path.join(dataset, 'List_of_testing_videos.txt')

# Extrating frames

In [6]:
#create file paths for storing frames
os.makedirs('frames/real',exist_ok=True)
os.makedirs('frames/fake',exist_ok=True)

In [7]:
#extracting faces function
def extract_frames(video_path,output_file, interval=30):
    cap=cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % interval == 0:
            frame_path = os.path.join(output_file, f"{os.path.basename(video_path).split('.')[0]}_frame_{count}.jpg")
            cv2.imwrite(frame_path,frame)
        count+=1
    cap.release()
    

    

            
            
        
        

In [4]:
#Now extracting frames from real videos
for video in os.listdir(real_videos_dir):
    extract_frames(os.path.join(real_videos_dir,video), 'frames/real')

In [5]:
#Now extracting frames from Youtube real videos
for video in os.listdir(youtube_real_dir):
    extract_frames(os.path.join(youtube_real_dir,video), 'frames/real')

In [6]:
#Extracting frames from fake videos
for video in os.listdir(fake_videos_dir):
    extract_frames(os.path.join(fake_videos_dir,video), 'frames/fake')

# Splittig train test dataset

In [8]:
import random

#getting list of all extracted frame paths

real_frames_path = [os.path.join('frames/real',file) for file in os.listdir('frames/real')]
fake_frames_path = [os.path.join('frames/fake',file) for file in os.listdir('frames/fake')]

#Creating labels: Real -> 0, Fake -> 1

real_labels = [0] * len(real_frames_path)
fake_labels = [1] * len(fake_frames_path)

#Merge Real and Fake data

all_frames = real_frames_path + fake_frames_path
all_labels = real_labels + fake_labels

#shuffle the merged data.

combined_data = list(zip(all_frames,all_labels))
random.shuffle(combined_data)
all_frames,all_labels = zip(*combined_data)

#split train and test data with 75%-25%

split_index = int(0.75 * len(all_frames))
xtrain,ytrain = all_frames[:split_index], all_labels[:split_index]
xtest, ytest = all_frames[split_index:], all_labels[split_index:]

In [9]:
print(len(all_frames))
print(len(real_frames_path))
print(len(fake_frames_path))
print(len(xtrain),len(ytrain))
print(len(xtest),len(ytest))

85877
12394
73483
64407 64407
21470 21470


In [9]:
import os
import random
import shutil

# Create train and test directories
os.makedirs('data/train/real')
os.makedirs('data/train/fake')
os.makedirs('data/test/real')
os.makedirs('data/test/fake')

# Function to copy files to the appropriate directory
def copy_files(file_paths, labels, base_dir):
    for file_path, label in zip(file_paths, labels):
        if label == 0:
            dest_dir = os.path.join(base_dir, 'real')
        else:
            dest_dir = os.path.join(base_dir, 'fake')
        shutil.copy(file_path, dest_dir)

# Copy training data
copy_files(xtrain, ytrain, 'data/train')

# Copy testing data
copy_files(xtest, ytest, 'data/test')

## Data Loading and Preprocessing

In [10]:
# Define transformations
transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Custom Dataset Class
class FrameDataset(Dataset):
    def __init__(self, root, transform=None):
        self.dataset = ImageFolder(root=root)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path, label = self.dataset.imgs[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = np.array(image)
            image = self.transform(image=image)["image"]
        return image, label

# Create datasets and dataloaders
train_dataset = FrameDataset(root='data/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=48, shuffle=True, num_workers= 0)
test_dataset = FrameDataset(root= "data/test", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=48, shuffle=True, num_workers= 0)


print(len(train_dataset),len(test_dataset))

80542 21685


# Load and Define Models

In [11]:
#New edited code with layers

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cuda:0")

class EfficientNetB4(nn.Module):
    def __init__(self):
        super(EfficientNetB4, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b4')
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(self.model._fc.in_features, 512)
        self.bn = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 2)
        self.model._fc = nn.Identity()  # Remove the original fully connected layer

    def forward(self, x):
        x = self.model(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.bn(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
class EfficientNetAutoAttB4(nn.Module):
    def __init__(self):
        super(EfficientNetAutoAttB4, self).__init__()
        self.base_model = EfficientNet.from_pretrained('efficientnet-b4')
        self.attention_layer = nn.MultiheadAttention(embed_dim=self.base_model._fc.in_features, num_heads=8, batch_first=True)
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(self.base_model._fc.in_features, 512)
        self.bn = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 2)
        self.base_model._fc = nn.Identity()  # Remove the original fully connected layer

    def forward(self, x):
        x = self.base_model.extract_features(x)  # Shape: [batch_size, channels, height, width]
        x = x.permute(0, 2, 3, 1)  # Shape: [batch_size, height, width, channels]
        x = x.reshape(x.size(0), -1, x.size(-1))  # Shape: [batch_size, seq_length, embed_dim]
        attn_output, _ = self.attention_layer(x, x, x)
        x = attn_output.mean(dim=1)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.bn(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    


# class XceptionModel(nn.Module):
#     def __init__(self):
#         super(XceptionModel, self).__init__()
#         self.model = timm.create_model('xception', pretrained=True, num_classes=1000)
#         self.dropout = nn.Dropout(p=0.5)
#         self.fc1 = nn.Linear(self.model.num_features, 512)
#         self.bn = nn.BatchNorm1d(512)
#         self.fc2 = nn.Linear(512, 2)
#         self.model.fc = nn.Identity()  # Remove the original fully connected layer

#     def forward(self, x):
#         x = self.model.forward_features(x)
#         x = x.view(x.size(0), -1)  # Flatten the tensor
#         x = self.dropout(x)
#         x = self.fc1(x)
#         x = self.bn(x)
#         x = nn.ReLU()(x)
#         x = self.dropout(x)
#         x = self.fc2(x)
#         return x
    
# Instantiate models
efficientnetb4 = EfficientNetB4()
efficientnetattb4 = EfficientNetAutoAttB4()


# Move models to device
efficientnetb4 = efficientnetb4.to(device)
efficientnetattb4 = efficientnetattb4.to(device)


Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4


In [12]:
#Xception Model
class XceptionModel(nn.Module):
    def __init__(self):
        super(XceptionModel, self).__init__()
        self.model = timm.create_model('xception', pretrained=True, num_classes=1000)
        self.model.fc = nn.Identity()  # Remove the original fully connected layer

        # Calculate the number of features output by the forward_features method
        with torch.no_grad():
            dummy_input = torch.zeros(1, 3, 224, 224)
            dummy_output = self.model.forward_features(dummy_input)
            num_features = dummy_output.view(dummy_output.size(0), -1).size(1)

        self.dropout = nn.Dropout(p=0.5)
        self.bn = nn.BatchNorm1d(num_features)
        self.fc2 = nn.Linear(num_features, 2)

    def forward(self, x):
        x = self.model.forward_features(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.dropout(x)
        x = self.bn(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x
    
#instantiate model
xception = XceptionModel()

#model to device
xception = xception.to(device)

# Training Loop with Mixed Precision and GradientScaling

In [13]:
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizers
optimizer_b4 = optim.Adam(efficientnetb4.parameters(), lr=0.0001)  #lr= 1e-4 
optimizer_attb4 = optim.Adam(efficientnetattb4.parameters(), lr=0.0001)
optimizer_xception = optim.Adam(xception.parameters(), lr=0.0001)



# Gradient Scaler for mixed precision
scaler = GradScaler()

# Function to calculate accuracy
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return torch.sum(preds == labels.data).item()

# Training function with gradient accumulation and mixed precision
def train_model(model, dataloader, criterion, optimizer, num_epochs=8, accumulation_steps=2):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_preds = 0
        total_samples = 0

        optimizer.zero_grad()
        
        for i, (inputs, labels) in enumerate(tqdm(dataloader)):
            inputs, labels = inputs.to(device), labels.to(device)

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss = loss / accumulation_steps

            scaler.scale(loss).backward()

            if (i + 1) % accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            running_loss += loss.item() * inputs.size(0)
            correct_preds += calculate_accuracy(outputs, labels)
            total_samples += inputs.size(0)

            # Memory cleanup
            del inputs, labels, outputs
            torch.cuda.empty_cache()

        epoch_loss = running_loss / total_samples
        epoch_acc = correct_preds / total_samples * 100
        print(f'Epoch {epoch}/{num_epochs-1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

    return model

In [23]:
torch.cuda.current_device()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2060 SUPER'

In [26]:
# Train models
#Train EfficientNetB4 Model
print('EfficientNetB4 Model Training...\n')
efficientnetb4 = train_model(efficientnetb4, train_loader, criterion, optimizer_b4, num_epochs=5)


EfficientNetB4 Model Training...



100%|██████████████████████████████████████████████████████████████████████████████| 1678/1678 [35:25<00:00,  1.27s/it]


Epoch 0/4, Loss: 0.1388, Accuracy: 89.32%


100%|██████████████████████████████████████████████████████████████████████████████| 1678/1678 [35:19<00:00,  1.26s/it]


Epoch 1/4, Loss: 0.0595, Accuracy: 95.62%


...

Epoch 2/4, Loss: 0.0373, Accuracy: 97.24%


100%|██████████████████████████████████████████████████████████████████████████████| 1678/1678 [31:00<00:00,  1.11s/it]


Epoch 3/4, Loss: 0.0274, Accuracy: 98.01%


100%|██████████████████████████████████████████████████████████████████████████████| 1678/1678 [33:07<00:00,  1.18s/it]

Epoch 4/4, Loss: 0.0213, Accuracy: 98.52%


In [24]:
#EfficientNetAutoAttB4 model
print('EfficientNetAutoAttB4 Model training...\n')
efficientnetattb4 = train_model(efficientnetattb4, train_loader, criterion, optimizer_attb4, num_epochs=5)



EfficientNetAutoAttB4 Model training...



100%|██████████████████████████████████████████████████████████████████████████████| 1678/1678 [46:35<00:00,  1.67s/it]


Epoch 0/4, Loss: 0.1250, Accuracy: 90.82%


100%|██████████████████████████████████████████████████████████████████████████████| 1678/1678 [26:26<00:00,  1.06it/s]


Epoch 1/4, Loss: 0.0577, Accuracy: 95.84%


...

Epoch 2/4, Loss: 0.0378, Accuracy: 97.22%


100%|██████████████████████████████████████████████████████████████████████████████| 1678/1678 [23:27<00:00,  1.19it/s]


Epoch 3/4, Loss: 0.0266, Accuracy: 98.06%


100%|██████████████████████████████████████████████████████████████████████████████| 1678/1678 [22:41<00:00,  1.23it/s]

Epoch 4/4, Loss: 0.0210, Accuracy: 98.52%


In [25]:
#XceptionNet Model Training
print('Xception Model Training...')
xception = train_model(xception, train_loader, criterion, optimizer_xception, num_epochs=5)

Xception Model Training...


100%|██████████████████████████████████████████████████████████████████████████████| 1678/1678 [13:58<00:00,  2.00it/s]


Epoch 0/4, Loss: 0.1343, Accuracy: 90.21%


100%|██████████████████████████████████████████████████████████████████████████████| 1678/1678 [13:54<00:00,  2.01it/s]


Epoch 1/4, Loss: 0.0635, Accuracy: 95.24%


...

Epoch 2/4, Loss: 0.0417, Accuracy: 96.87%


100%|██████████████████████████████████████████████████████████████████████████████| 1678/1678 [13:53<00:00,  2.01it/s]


Epoch 3/4, Loss: 0.0311, Accuracy: 97.72%


100%|██████████████████████████████████████████████████████████████████████████████| 1678/1678 [14:27<00:00,  1.93it/s]

Epoch 4/4, Loss: 0.0236, Accuracy: 98.29%


# Save Trained Models

In [14]:
 # Save model weights
# torch.save(efficientnetb4.state_dict(), 'efficientnetb4.pth')
# torch.save(efficientnetattb4.state_dict(), 'efficientnetattb4.pth')
# torch.save(xception.state_dict(), 'xception.pth')

torch.save({
    'efficientnetb4_state_dict': efficientnetb4.state_dict(),
    'efficientnetattb4_state_dict': efficientnetattb4.state_dict(),
    'xception_state_dict': xception.state_dict(),
}, 'models.h5')

print('Model weights are saved successfully.')

Model weights are saved successfully.


In [15]:
checkpoint = torch.load('models.h5')
efficientnetb4.load_state_dict(checkpoint['efficientnetb4_state_dict'])
efficientnetattb4.load_state_dict(checkpoint['efficientnetattb4_state_dict'])
xception.load_state_dict(checkpoint['xception_state_dict'])

print('Model weights are loaded successfully.')

Model weights are loaded successfully.


## Testing Models

In [30]:
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return torch.sum(preds == labels.data).item(), len(torch.unique(labels))

# Function to load model weights and test the model
def test_model(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct_preds = 0
    total_samples = 0
    total_classes = set()

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

#             running_loss += loss.item() * inputs.size(0)
#             correct_preds += calculate_accuracy(outputs, labels)
#             total_samples += inputs.size(0)
            
            running_loss += loss.item() * inputs.size(0)
            correct, unique_classes = calculate_accuracy(outputs, labels)
            correct_preds += correct
            total_samples += inputs.size(0)
            total_classes.update(labels.cpu().numpy())

            # Memory cleanup
            del inputs, labels, outputs
            torch.cuda.empty_cache()

    loss = running_loss / total_samples
    accuracy = correct_preds / total_samples * 100
    total_classes_detected = len(total_classes)
    
    print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    print(f'Total Classes Detected: {total_classes_detected}')
    print(f'Total Samples: {total_samples}')
    print(f'True Validation Samples (Correct Predictions): {correct_preds}')

# # Load model weights
# efficientnetb4.load_state_dict(torch.load('efficientnetb4.pth'))
# efficientnetattb4.load_state_dict(torch.load('efficientnetattb4.pth'))
# xception.load_state_dict(torch.load('xception.pth'))

# Test models
print('Model Name: EfficientNetB4')
test_model(efficientnetb4, test_loader, criterion)
print('Model Name: EfficientNetAutoAttB4')
test_model(efficientnetattb4, test_loader, criterion)
print('Model Name: XceptionNet')
test_model(xception, test_loader, criterion)

Model Name: EfficientNetB4


100%|████████████████████████████████████████████████████████████████████████████████| 452/452 [09:50<00:00,  1.31s/it]


Test Loss: 0.0293, Test Accuracy: 99.05%
Total Classes Detected: 2
Total Samples: 21685
True Validation Samples (Correct Predictions): 21479
Model Name: EfficientNetAutoAttB4


100%|████████████████████████████████████████████████████████████████████████████████| 452/452 [03:34<00:00,  2.11it/s]


Test Loss: 0.0330, Test Accuracy: 98.76%
Total Classes Detected: 2
Total Samples: 21685
True Validation Samples (Correct Predictions): 21417
Model Name: XceptionNet


100%|████████████████████████████████████████████████████████████████████████████████| 452/452 [03:12<00:00,  2.35it/s]

Test Loss: 0.1019, Test Accuracy: 97.51%
Total Classes Detected: 2
Total Samples: 21685
True Validation Samples (Correct Predictions): 21145


## Model Essembling and Inferencing 

In [31]:
import torch
from torch import nn
from tqdm import tqdm

# Assuming models are already defined and loaded as efficientnetb4, efficientnetattb4, and xception

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def ensemble_models(models, inputs):
    outputs = [model(inputs) for model in models]
    avg_output = torch.mean(torch.stack(outputs), dim=0)
    return avg_output

def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return torch.sum(preds == labels.data).item(), len(torch.unique(labels))

def test_ensemble_model(models, dataloader, criterion):
    for model in models:
        model.eval()
        
    running_loss = 0.0
    correct_preds = 0
    total_samples = 0
    total_classes = set()

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = ensemble_models(models, inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            correct, unique_classes = calculate_accuracy(outputs, labels)
            correct_preds += correct
            total_samples += inputs.size(0)
            total_classes.update(labels.cpu().numpy())

            # Memory cleanup
            del inputs, labels, outputs
            torch.cuda.empty_cache()

    loss = running_loss / total_samples
    accuracy = correct_preds / total_samples * 100
    total_classes_detected = len(total_classes)
    
    print(f'Ensemble Test Loss: {loss:.4f}, Ensemble Test Accuracy: {accuracy:.2f}%')
    print(f'Total Classes Detected: {total_classes_detected}')
    print(f'Total Samples: {total_samples}')
    print(f'True Validation Samples (Correct Predictions): {correct_preds}')

# Test the ensemble model
models = [efficientnetb4, efficientnetattb4, xception]
print('Ensemble Model')
test_ensemble_model(models, test_loader, criterion)


Ensemble Model


100%|████████████████████████████████████████████████████████████████████████████████| 452/452 [05:37<00:00,  1.34it/s]

Ensemble Test Loss: 0.0233, Ensemble Test Accuracy: 99.41%
Total Classes Detected: 2
Total Samples: 21685
True Validation Samples (Correct Predictions): 21558


## Model Inferencing

In [32]:
import torch

def ensemble_predict(models, input_image):
    outputs = [model(input_image.to(device)) for model in models]
    avg_output = torch.mean(torch.stack(outputs), dim=0)
    _, predicted = torch.max(avg_output, 1)
    return predicted

In [16]:
import tkinter as tk
from tkinter import filedialog, messagebox

# Preprocess frame
def preprocess_frame(frame_path):
    transform = A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
    image = Image.open(frame_path)
    image = np.array(image)
    image = transform(image=image)['image']
    image = image.unsqueeze(0)
    return image

# Ensemble predict
def ensemble_predict(models, input_image):
    outputs = [model(input_image) for model in models]
    avg_output = torch.mean(torch.stack(outputs), dim=0)
    prediction = torch.argmax(avg_output, dim=1)
    return prediction

# Extract frames
def extract_frames(video_path, output_folder, interval=30):
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % interval == 0:
            frame_path = os.path.join(output_folder, f"frame_{count}.jpg")
            cv2.imwrite(frame_path, frame)
        count += 1
    cap.release()
    return count // interval  # Number of extracted frames

# Process video
def process_video(video_path, models):
    frame_folder = 'temp_frames'
    os.makedirs(frame_folder, exist_ok=True)
    
    # Extract frames from the video
    num_frames = extract_frames(video_path, frame_folder)
    
    # Predict on each frame
    real_count, fake_count = 0, 0
    for i in range(num_frames):
        frame_path = os.path.join(frame_folder, f'frame_{i * 30}.jpg')  # Considering the interval used in extract_frames
        if os.path.exists(frame_path):
            input_image = preprocess_frame(frame_path)
            prediction = ensemble_predict(models, input_image)
            if prediction.item() == 0:
                real_count += 1
            else:
                fake_count += 1
    
    # Determine the overall result based on frame predictions
    result = "Real Video" if real_count > fake_count else "Fake Video"
    
    # Clean up temporary frames
    for frame_file in os.listdir(frame_folder):
        os.remove(os.path.join(frame_folder, frame_file))
    os.rmdir(frame_folder)
    
    return result

# Tkinter interface
def upload_video():
    video_path = filedialog.askopenfilename(filetypes=[("Video files", ".mp4;.avi;*.mov")])
    if video_path:
        result = process_video(video_path, models)
        messagebox.showinfo("Result", f"The uploaded video is a {result}")

# Create Tkinter window
root = tk.Tk()
root.title("Deepfake Detection")
root.geometry("300x150")

# Upload button
upload_button = tk.Button(root, text="Upload Video", command=upload_video)
upload_button.pack(expand=True)

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Yousuf Traders\AppData\Local\Temp\ipykernel_9096\146582387.py", line 73, in upload_video
    result = process_video(video_path, models)
                                       ^^^^^^
NameError: name 'models' is not defined
